# 3 Pointers Made against game_totals.csv

### Import packages

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
from scipy.stats import pearsonr
import itertools

pd.set_option("display.max_columns", None)

### Set working directory

In [2]:
# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

# Change working directory
os.chdir('/Users/tyler/OneDrive/Documents/Python/NBA')

# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA\backend\analysis\3p
Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA


## Exploratory Data Analysis

### Import data

In [3]:
df = pd.read_csv('backend/data/totals/game_totals.csv').drop(['Unnamed: 0'], axis=1)

### Basic exploration

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40376 entries, 0 to 40375
Data columns (total 23 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     40376 non-null  object 
 1   visitor  40376 non-null  object 
 2   home     40376 non-null  object 
 3   team     40376 non-null  int64  
 4   fg       40374 non-null  float64
 5   fga      40374 non-null  float64
 6   fg_perc  40374 non-null  float64
 7   3p       40374 non-null  float64
 8   3pa      40374 non-null  float64
 9   3p_perc  40374 non-null  float64
 10  ft       40374 non-null  float64
 11  fta      40374 non-null  float64
 12  ft_perc  40374 non-null  float64
 13  orb      40374 non-null  float64
 14  drb      40374 non-null  float64
 15  trb      40374 non-null  float64
 16  ast      40374 non-null  float64
 17  stl      40374 non-null  float64
 18  blk      40374 non-null  float64
 19  tov      40374 non-null  float64
 20  pf       40374 non-null  float64
 21  pts      403

In [5]:
df.head()

,date,visitor,home,team,fg,fga,fg_perc,3p,3pa,3p_perc,ft,fta,ft_perc,orb,drb,trb,ast,stl,blk,tov,pf,pts,starter
0,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,0,39.0,79.0,0.494,7.0,13.0,0.538,23.0,32.0,0.719,13.0,36.0,49.0,22.0,10.0,5.0,16.0,21.0,108.0,NaN
1,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,1,25.0,65.0,0.385,3.0,17.0,0.176,13.0,22.0,0.591,4.0,25.0,29.0,13.0,6.0,3.0,21.0,24.0,66.0,NaN
2,"Tue, Oct 31, 2006",Phoenix Suns,Los Angeles Lakers,0,40.0,77.0,0.519,13.0,30.0,0.433,13.0,17.0,0.765,4.0,25.0,29.0,29.0,7.0,5.0,21.0,25.0,106.0,NaN
3,"Tue, Oct 31, 2006",Phoenix Suns,Los Angeles Lakers,1,46.0,83.0,0.554,6.0,12.0,0.500,16.0,24.0,0.667,12.0,31.0,43.0,30.0,11.0,1.0,20.0,19.0,114.0,NaN
4,"Wed, Nov 1, 2006",Indiana Pacers,Charlotte Bobcats,0,35.0,86.0,0.407,6.0,16.0,0.375,30.0,41.0,0.732,18.0,32.0,50.0,26.0,14.0,5.0,17.0,26.0,106.0,NaN


In [6]:
# Convert 'date' column to Date object
df['date'] = pd.to_datetime(df['date'])


# Conver 'team' column to Team Name
df['team'] = np.where(df['team'], df['home'], df['visitor'])


# Merge in opponents (see team defensive stats)
df = pd.merge(
    df, 
    df, 
    left_on=['date', 'visitor', 'home'], 
    right_on=['date', 'visitor', 'home'],
    suffixes=('', '_opp'),
    how='left')

df = df[df['team'] != df['team_opp']]


df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40376 entries, 1 to 80750
Data columns (total 43 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         40376 non-null  datetime64[ns]
 1   visitor      40376 non-null  object        
 2   home         40376 non-null  object        
 3   team         40376 non-null  object        
 4   fg           40374 non-null  float64       
 5   fga          40374 non-null  float64       
 6   fg_perc      40374 non-null  float64       
 7   3p           40374 non-null  float64       
 8   3pa          40374 non-null  float64       
 9   3p_perc      40374 non-null  float64       
 10  ft           40374 non-null  float64       
 11  fta          40374 non-null  float64       
 12  ft_perc      40374 non-null  float64       
 13  orb          40374 non-null  float64       
 14  drb          40374 non-null  float64       
 15  trb          40374 non-null  float64       
 16  ast 

# Dataframe of team's last 15 performances

In [7]:
# Return ten lastest dates team played
def last_15_date(team, date):
    schedule = df[df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 15 < 0:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        date_11, date_12 = schedule.iloc[date_index - 11]['date'], schedule.iloc[date_index - 12]['date']
        date_13, date_14 = schedule.iloc[date_index - 13]['date'], schedule.iloc[date_index - 14]['date']
        date_15 = schedule.iloc[date_index - 15]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10, date_11, date_12, date_13, date_14, date_15

df['dates'] = df.apply(lambda x: last_15_date(x.team, x.date), axis=1)
df['date_1'], df['date_2'] = df['dates'].apply(lambda x: x[0]), df['dates'].apply(lambda x: x[1])
df['date_3'], df['date_4'] = df['dates'].apply(lambda x: x[2]), df['dates'].apply(lambda x: x[3])
df['date_5'], df['date_6'] = df['dates'].apply(lambda x: x[4]), df['dates'].apply(lambda x: x[5])
df['date_7'], df['date_8'] = df['dates'].apply(lambda x: x[6]), df['dates'].apply(lambda x: x[7])
df['date_9'], df['date_10'] = df['dates'].apply(lambda x: x[8]), df['dates'].apply(lambda x: x[9])
df['date_11'], df['date_12'] = df['dates'].apply(lambda x: x[10]), df['dates'].apply(lambda x: x[11])
df['date_13'], df['date_14'] = df['dates'].apply(lambda x: x[12]), df['dates'].apply(lambda x: x[13])
df['date_15'] = df['dates'].apply(lambda x: x[14])

In [8]:
# Statistics
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts',
         'fg_opp', 'fga_opp', '3p_opp', '3pa_opp', 'ft_opp', 'fta_opp', 'orb_opp', 'drb_opp', 'trb_opp', 'ast_opp', 
         'stl_opp', 'blk_opp', 'tov_opp', 'pf_opp', 'pts_opp']

# X and y column names to merge on
x_cols = ['date', 'team'] + stats

last_15_games = df.copy()
last_15_games.loc[:, 'target'] = last_15_games.loc[:, '3p']
X = df[x_cols].copy()

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

In [9]:
# Calculate z-score
def z_score(value, mean, std):
    return (value - mean) / std

### Last 15 Performances (Unweighted)

In [11]:
# Statistics
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts',
         'fg_opp', 'fga_opp', '3p_opp', '3pa_opp', 'ft_opp', 'fta_opp', 'orb_opp', 'drb_opp', 'trb_opp', 'ast_opp', 
         'stl_opp', 'blk_opp', 'tov_opp', 'pf_opp', 'pts_opp']

dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']

last_15 = last_15_games.copy()

# Calculate mean for each stat over a team's last performance
for stat in stats:
    last_15[stat + '_last_15'] = 0
    last_15[stat + '_last_3'] = 0
    last_15[stat + '_last_1'] = 0
    
    for date in dates:
        # Last 15 games
        last_15[stat + '_last_15'] = last_15[stat + '_last_15'] + last_15[stat + date]
        
        # Last 3 games
        if date in ['_1', '_2', '_3']:
            last_15[stat + '_last_3'] = last_15[stat + '_last_3'] + last_15[stat + date]
        
        # Last game
        if date in ['_1']:
            last_15[stat + '_last_1'] = last_15[stat + '_last_1'] + last_15[stat + date]
        
    
    last_15[stat + '_last_15'] = last_15[stat + '_last_15'] / 15
    last_15[stat + '_last_3'] = last_15[stat + '_last_3'] / 3

# Calculate difference between last 15 games, 3 games and last game
for stat in stats:
    last_15[stat + '_trend_3'] = last_15[stat + '_last_15'] - last_15[stat + '_last_3']
    last_15[stat + '_trend_1'] = last_15[stat + '_last_15'] - last_15[stat + '_last_1']

# Sum stats for opposing teams for each game
last_15 = last_15.groupby(['date', 'visitor', 'home']).sum()

# Keep columns
stats_15 = [stat + '_last_15' for stat in stats]
stats_3 = [stat + '_last_3' for stat in stats]
stats_1 = [stat + '_last_1' for stat in stats]
trend_3 = [stat + '_trend_3' for stat in stats]
trend_1 = [stat + '_trend_1' for stat in stats]
last_15 = last_15[['target'] + stats_1 + stats_3 + stats_15 + trend_1 + trend_3]

# Calculate percentages
last_15['fg_perc_last_15'] = last_15['fg_last_15'] / last_15['fga_last_15']
last_15['fg_perc_opp_last_15'] = last_15['fg_opp_last_15'] / last_15['fga_opp_last_15']

last_15['3p_perc_last_15'] = last_15['3p_last_15'] / last_15['3pa_last_15']
last_15['3p_perc_opp_last_15'] = last_15['3p_opp_last_15'] / last_15['3pa_opp_last_15']

last_15['ft_perc_last_15'] = last_15['ft_last_15'] / last_15['fta_last_15']
last_15['ft_perc_opp_last_15'] = last_15['ft_opp_last_15'] / last_15['fta_opp_last_15']

# Calculate advanced stats
last_15['ts_perc_last_15'] = last_15['pts_last_15'] / (2 * (last_15['fga_last_15'] + .44 * last_15['fta_last_15']))
last_15['ts_perc_opp_last_15'] = last_15['pts_opp_last_15'] / (2 * (last_15['fga_opp_last_15'] + .44 * last_15['fta_opp_last_15']))

last_15['efg_perc_last_15'] = (last_15['fg_last_15'] + (.5 * last_15['3p_last_15'])) / last_15['fga_last_15']
last_15['efg_perc_opp_last_15'] = (last_15['fg_opp_last_15'] + (.5 * last_15['3p_opp_last_15'])) / last_15['fga_opp_last_15']

last_15['3par_last_15'] = last_15['3pa_last_15'] / last_15['fga_last_15']
last_15['3par_opp_last_15'] = last_15['3pa_opp_last_15'] / last_15['fga_opp_last_15']

last_15['ftr_last_15'] = last_15['fta_last_15'] / last_15['fga_last_15']
last_15['ftr_opp_last_15'] = last_15['fta_opp_last_15'] / last_15['fga_opp_last_15']

last_15['orb_perc_last_15'] = last_15['orb_last_15'] / (last_15['orb_last_15'] + last_15['drb_opp_last_15'])
last_15['orb_perc_opp_last_15'] = last_15['orb_opp_last_15'] / (last_15['orb_opp_last_15'] + last_15['drb_last_15'])

last_15['drb_perc_last_15'] = last_15['drb_last_15'] / (last_15['drb_last_15'] + last_15['orb_opp_last_15'])
last_15['drb_perc_opp_last_15'] = last_15['drb_opp_last_15'] / (last_15['drb_opp_last_15'] + last_15['orb_last_15'])

last_15['trb_perc_last_15'] = last_15['trb_last_15'] / (last_15['trb_last_15'] + last_15['trb_opp_last_15'])
last_15['trb_perc_opp_last_15'] = last_15['trb_opp_last_15'] / (last_15['trb_opp_last_15'] + last_15['trb_last_15'])

last_15['ast_perc_last_15'] = last_15['ast_last_15'] / last_15['fg_last_15']
last_15['ast_perc_opp_last_15'] = last_15['ast_opp_last_15'] / last_15['fg_opp_last_15']

last_15 = last_15.dropna(axis=0)
last_15.tail(5)

target  fg_last_1  \
date       visitor                home                                      
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers     21.0       77.0   
           Phoenix Suns           Houston Rockets         29.0       88.0   
           Portland Trail Blazers New York Knicks         27.0       83.0   
           Toronto Raptors        Los Angeles Clippers    22.0       82.0   
2022-03-17 Detroit Pistons        Orlando Magic            0.0       73.0   

                                                        fga_last_1  3p_last_1  \
date       visitor                home                                          
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers        170.0       27.0   
           Phoenix Suns           Houston Rockets            167.0       30.0   
           Portland Trail Blazers New York Knicks            171.0       21.0   
           Toronto Raptors        Los Angeles Clippers       194.0       22.0   
2022-03-17 Detroit Pistons        Orlando Magic              175.0       25.0   

                                                        3pa_last_1  ft_last_1  \
date       visitor                home                                          
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         63.0       49.0   
           Phoenix Suns           Houston Rockets             65.0       30.0   
           Portland Trail Blazers New York Knicks             53.0       33.0   
           Toronto Raptors        Los Angeles Clippers        73.0       39.0   
2022-03-17 Detroit Pistons        Orlando Magic               75.0       35.0   

                                                        fta_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         64.0   
           Phoenix Suns           Houston Rockets             40.0   
           Portland Trail Blazers New York Knicks             42.0   
           Toronto Raptors        Los Angeles Clippers        53.0   
2022-03-17 Detroit Pistons        Orlando Magic               49.0   

                                                        orb_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         23.0   
           Phoenix Suns           Houston Rockets             15.0   
           Portland Trail Blazers New York Knicks             22.0   
           Toronto Raptors        Los Angeles Clippers        31.0   
2022-03-17 Detroit Pistons        Orlando Magic               17.0   

                                                        drb_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         74.0   
           Phoenix Suns           Houston Rockets             58.0   
           Portland Trail Blazers New York Knicks             74.0   
           Toronto Raptors        Los Angeles Clippers        76.0   
2022-03-17 Detroit Pistons        Orlando Magic               58.0   

                                                        trb_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         97.0   
           Phoenix Suns           Houston Rockets             73.0   
           Portland Trail Blazers New York Knicks             96.0   
           Toronto Raptors        Los Angeles Clippers       107.0   
2022-03-17 Detroit Pistons        Orlando Magic               75.0   

                                                        ast_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         53.0   
           Phoenix Suns           Houston Rockets             59.0   
           Portland Trail Blazers New York Knicks             52.0   
           Toronto Raptors        Los Angeles Clippers        53.0

## Correlations

In [12]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in last_15:
    corr_p = pearsonr(last_15['target'], last_15[col])
    row = {'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print correlations
corr_df = corr_df[corr_df['p-value'] < .05].drop(['p-value'], axis=1).sort_values(['corr'], axis=0, ascending=False)
corr_df

,corr,stat
0,1.00,target
63,0.74,3p_last_15
161,0.74,3par_last_15
64,0.74,3pa_last_15
34,0.73,3pa_last_3
...,...,...
67,-0.29,orb_last_15
163,-0.39,ftr_last_15
164,-0.42,ftr_opp_last_15
166,-0.46,orb_perc_opp_last_15


## Save dataframe with significantly correlated stats

In [13]:
stats = corr_df[corr_df['corr'].abs() >= .6]['stat']
df = last_15[stats]

corr_df[corr_df['corr'].abs() >= .6]

# df.to_csv('backend/data/inputs/3p/game_totals.csv')

,corr,stat
0,1.00,target
63,0.74,3p_last_15
161,0.74,3par_last_15
64,0.74,3pa_last_15
34,0.73,3pa_last_3
79,0.70,3pa_opp_last_15
33,0.69,3p_last_3
4,0.69,3pa_last_1
162,0.68,3par_opp_last_15
78,0.68,3p_opp_last_15
